In [1]:
#Imports
import numpy as np
import pandas as pd

import json
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from IPython.core.display import display, HTML

import HTMLSentenceTokenizer
sentence = HTMLSentenceTokenizer.HTMLSentenceTokenizer()
from nltk.tokenize import word_tokenize
import ast
import pickle

from tqdm import tnrange, tqdm_notebook

import functions 
clean = functions.clean()

In [5]:
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/{}"

In [3]:
#Open cases
path = "/Users/conorosully/Documents/Legal-Case-Prediction/data/clean/{}.json"

with open(path.format('text_alpha'), 'r') as readfile:
    text = json.load(readfile)
    readfile.close()

IDs = text.keys()
print(len(IDs))
print(list(text.keys())[0])
print(len(text['HUDOC-ECHR-2012-001-110881']))


8703
HUDOC-ECHR-1999-001-58225
7


In [4]:
att = pd.read_csv("/Users/conorosully/Documents/Legal-Case-Prediction/data/preclean/case_attributes.csv",'#')
print(att.columns)
print(len(att))
IDs = att['id']
att.head(1)

Index(['id', 'type', 'url', 'doc-name', 'doc-id-label', 'doc-date', 'vl-type',
       'created-at', 'updated-at', 'app-no-parts', 'app-nos', 'articles',
       'conclusions', 'courts', 'ecli', 'judgement-date', 'judges',
       'last-modified-time', 'nonviolations', 'organisations', 'respondents',
       'separate-opinion', 'violations'],
      dtype='object')
55652


,id,type,url,doc-name,doc-id-label,doc-date,vl-type,created-at,updated-at,app-no-parts,...,courts,ecli,judgement-date,judges,last-modified-time,nonviolations,organisations,respondents,separate-opinion,violations
0,HUDOC-ECHR-1959-001-49198,eu-chrs,http://hudoc.echr.coe.int/app/conversion/docx/...,greece <br>against<br> the united kingdom,176/56,1959-04-20,eu_chr,2017-09-18T10:01:36.112Z,2018-10-26T16:53:58.954Z,"['176', '56']",...,NaN,NaN,NaN,NaN,2018-10-07,NaN,['Committee of Ministers'],['GBR'],NaN,NaN


In [7]:
IDs[27805]

'HUDOC-ECHR-2009-001-96300'

In [8]:
ID = 'HUDOC-ECHR-2009-001-96300'
doc = clean.getCase(ID)[ID]
print(type(doc))
#print(doc)
#clean.renderHTML(ID)

<class 'str'>


In [6]:
IDs = IDs.drop(27805)
len(IDs)

55651

In [13]:
with open(path.format('preclean/documents.pickle'), 'wb') as f:
    
    for ID in tqdm_notebook(IDs):
        
        doc = clean.getCase(ID)[ID]

        tokens = word_tokenize(doc)
        tokens = [word for word in tokens if word.isalpha()]
        tokens = [w.lower() for w in tokens]
        
        pickle.dump(tokens,f)
                
f.close()


## Create Embeddings

In [9]:
#To load from pickle file
documents = []
count = 0
with open(path.format('preclean/documents2.pickle'), 'rb') as fr:
    try:
        while True:
            documents.append(pickle.load(fr))
            if count%5000 == 0:
                print(count, end=" ")
            count += 1
    except EOFError:
        pass

print(len(documents))
#print(documents[0])

0 5000 10000 15000 20000 25000 30000 35000 40000 45000 50000 55000 55651


In [10]:
#Alternative
documents = []
for ID in tqdm_notebook(IDs):
    doc = text[ID]['full']
    tokens = word_tokenize(doc)
    documents.append(tokens)
print(len(documents))


8703


We use these hyper-parameter values for WIKI (APNEWS): vector size = 300 (300), window size = 15 (15), min count = 20 (10), sub-sampling threshold = 10−5
(10−5),negative sample = 5, epoch = 20 (30).

In [10]:
documents = [TaggedDocument(doc, [i]) 
             for i, doc in tqdm_notebook(enumerate(documents))]

In [11]:
# define training data
for s in [100,200,300]:
    # train model
    model = Doc2Vec(documents, size=s, window=5, min_count=10, workers=4,epoch = 20)
    name = 'embeddings/doc2vec2_{}'.format(s)
    model.save(path.format(name))
    print(s)

/Users/conorosully/miniconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/Users/conorosully/miniconda3/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


100
200
300
